In [1]:
using Pipe
using MarketTechnicals
using Plots
# using TimeSeries
using DataFrames
using StatsPlots
using CSV

In [2]:
theme(:dark)
gr()

Plots.GRBackend()

In [3]:

# ta = readtimearray("../data/csv/BTCUSDT-15m.csv", format="yyyy-mm-dd HH:MM", delim=',', meta=nothing)
# taf = @pipe ta |> from(_, DateTime(2021, 3, 1)) |> to(_, DateTime(2021, 4, 1)) 
# df = DataFrame(taf)

In [97]:
# csv_df = DataFrame(CSV.File("../data/csv/BTCUSDT-15m.csv", dateformat="yyyy-mm-dd HH:MM"))
df = csv_df[(csv_df.time .>= DateTime(2021, 1, 1)) .& (csv_df.time .<= DateTime(2021, 4, 1)) , :]
3

3

In [98]:
period_ema_slow = 100
period_ema_fast = 21
period_ema_scalp = 14

insertcols!(df, :ema_slow => vec([
            fill(df.close[1], (period_ema_slow - 1,1))
            ema(df.close, period_ema_slow)
        ]), :ema_fast => vec([
            fill(df.close[1], (period_ema_fast - 1,1))
            ema(df.close, period_ema_fast)
        ]), :ema_scalp => vec([
            fill(df.close[1], (period_ema_scalp - 1,1))
            ema(df.close, period_ema_scalp)
        ]), 
)
2

2

In [99]:
insertcols!(df, 
    :log_ema_fast_slow => log.( df.ema_fast ./ df.ema_slow ) .* 1000, 
    :log_ema_scalp_slow => log.( df.ema_scalp ./ df.ema_slow ) .* 1000, 
    :log_ema_scalp_fast => log.( df.ema_scalp ./ df.ema_fast ) .* 1000, 
    :log_close_ema_scalp => log.( df.close ./ df.ema_scalp ) .* 1000, 
    :log_close_ema_fast => log.( df.close ./ df.ema_fast ) .* 1000, 
    :log_close_ema_slow => log.( df.close ./ df.ema_slow ) .* 1000, 
    
)
1
# log.( df.close ./ df.ema_fast ) .* 1000

1

In [100]:
nrows = nrow(df)

8630

In [101]:
insertcols!(df, 
    :reason => [fill("", (nrows,1))...],
    :price => [fill(0., (nrows,1))...],
    :buy => [fill(0., (nrows,1))...],
    :sell => [fill(0., (nrows,1))...],
    :pnl => [fill(0., (nrows,1))...],
    :capital => [fill(0., (nrows,1))...]
)
1

1

In [102]:
capital, riskPercent = 100., 0.01
isLong, priceIn, priceOut, buySize, pnl, stopLoss  = false, 0, 0, 0, 0, 0 
feePercent, stopLossReason = 0.001, "SL"
for r in eachrow(df)
    if isLong
        priceOut = r.close * (1 - feePercent)
        pnl = buySize * (priceOut - priceIn)
        if r.close < stopLoss # close SL
            capital += pnl
            isLong = false
#             priceOut = r.close * (1 - feePercent)
#             r.capital = capital
            r.pnl = pnl
            r.reason = "SL"
            r.sell = buySize
            r.price = priceOut
#             push!(dtrf, (r.timestamp, stopLossReason, 0, buySize, priceout, profit, capital))
        elseif r.close > priceIn * (1 + 0.05)
            capital += pnl
            isLong = false
#             priceOut = r.close * (1 - feePercent)
#             r.capital = capital
            r.pnl = pnl
            r.reason = "TP"
            r.sell = buySize
            r.price = priceOut
#             push!(dtrf, (r.timestamp, "TH", 0, buySize, priceOut, profit, capital))
        elseif r.close > priceIn * (1 + 0.005)
        end
    else #not in pos
        if r.log_close_ema_fast < -10 && r.log_ema_fast_slow > 5
            isLong = true
            position = capital * riskPercent
            priceIn = r.close * (1 + feePercent)
            stopLoss = r.close * (1 - 0.03)
            buySize = position / (priceIn - stopLoss)
#             println("priceIn: ", priceIn)
#             println("stoploss: ", stopLoss)
#             println("buySize: ", buySize)
            r.price = priceIn
            r.reason = "long"
            r.buy = buySize
            
#             push!(dtrf, (r.time, "LONG", buySize, 0, priceIn, 0, capital))
            slrsn = "SL"
        end
    end
    r.capital = capital
    # push!(dtrf, (r.timestamp, pricein, priceout, sizep, capt, islong))
end


In [103]:
using Query

In [104]:
trades = df |> 
    @filter(_.reason != "") |>
    @map({_.time,_.close, _.reason, _.buy, _.sell, _.price, _.pnl, _.capital}) |>
    DataFrame

# trades
# CSV.write("trades.csv", trades)
1
# = @from r in df begin
#      @where r.reason != ""
#      @select r
# end
# nothing # hide

1

In [105]:
CSV.write("trades.csv", trades)
trades[:,2:8]

,close,reason,buy,sell,price,pnl,capital
,Float64,String,Float64,Float64,Float64,Float64,Float64
1,28880.0,long,0.00111697,0.0,28908.9,0.0,100.0
2,30695.1,TP,0.0,0.00111697,30664.4,1.96082,101.961
3,30667.2,long,0.0010725,0.0,30697.9,0.0,101.961
4,32412.0,TP,0.0,0.0010725,32379.6,1.80362,103.764
5,33355.1,long,0.00100352,0.0,33388.4,0.0,103.764
6,32341.2,SL,0.0,0.00100352,32308.8,-1.08343,102.681
7,31718.7,long,0.00104427,0.0,31750.4,0.0,102.681
8,30682.1,SL,0.0,0.00104427,30651.4,-1.14766,101.533
9,34179.7,long,0.00095825,0.0,34213.9,0.0,101.533
